# Libraries

In [25]:
import os, json, requests, string, time, glob, aiohttp, asyncio, warnings, math
warnings.filterwarnings('ignore')
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from tqdm import tqdm
from tqdm.asyncio import tqdm as atqdm
from tqdm.notebook import tqdm as ntqdm

# Edit this section

In [23]:
thn_anggaran = '2024'
nama_file = 'sirup_2024'

# No need to edit below sections

In [24]:
per_page = '50000'

# Open the browser

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('w3c')
options.add_argument('--disable_gpu')
options.add_argument('--no-sandbox')
options.add_argument("--start-maximized")
options.add_argument("--window-size=1920,1080")
#options.add_argument("--headless")

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}
DRIVER = webdriver.Chrome(options=options)

In [4]:
initial_url = 'https://sirup.lkpp.go.id/sirup/caripaketctr/index'

In [5]:
DRIVER.get(initial_url)

In [6]:
S = requests.Session()
for cookie in DRIVER.get_cookies(): S.cookies.set(cookie['name'], cookie['value'])

In [13]:
headers = {
    'Accept': 'application/json, text/plain, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}

In [31]:
url_head = 'https://sirup.lkpp.go.id/sirup/caripaketctr/search?tahunAnggaran='+thn_anggaran+'&jenisPengadaan=&metodePengadaan=&minPagu=&maxPagu=&bulan=&lokasi=&kldi=&pdn=&ukm=&draw=2&columns%5B0%5D%5Bdata%5D=&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=false&columns%5B0%5D%5Borderable%5D=false&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=paket&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=pagu&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=jenisPengadaan&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=isPDN&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=isUMK&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=metode&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B7%5D%5Bdata%5D=pemilihan&columns%5B7%5D%5Bname%5D=&columns%5B7%5D%5Bsearchable%5D=true&columns%5B7%5D%5Borderable%5D=true&columns%5B7%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B7%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B8%5D%5Bdata%5D=kldi&columns%5B8%5D%5Bname%5D=&columns%5B8%5D%5Bsearchable%5D=true&columns%5B8%5D%5Borderable%5D=true&columns%5B8%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B8%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B9%5D%5Bdata%5D=satuanKerja&columns%5B9%5D%5Bname%5D=&columns%5B9%5D%5Bsearchable%5D=true&columns%5B9%5D%5Borderable%5D=true&columns%5B9%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B9%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B10%5D%5Bdata%5D=lokasi&columns%5B10%5D%5Bname%5D=&columns%5B10%5D%5Bsearchable%5D=true&columns%5B10%5D%5Borderable%5D=true&columns%5B10%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B10%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B11%5D%5Bdata%5D=id&columns%5B11%5D%5Bname%5D=&columns%5B11%5D%5Bsearchable%5D=true&columns%5B11%5D%5Borderable%5D=true&columns%5B11%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B11%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=5&order%5B0%5D%5Bdir%5D=DESC&search%5Bvalue%5D=&search%5Bregex%5D=false&start='
url_middle = '&length='
url_tail =  '&_=1723085212758'

# Get Total Rows

In [32]:
r = S.get(url_head+'0'+url_middle+'10'+url_tail, headers=headers)

In [33]:
total_rows = r.json()['recordsFiltered']

In [34]:
total_pages = math.ceil(total_rows/int(per_page))

In [35]:
total_rows, total_pages

(3864646, 78)

# Create temp dir

In [28]:
dir = 'sirup_json_data'
if not os.path.exists(dir): os.makedirs(dir)

# Download all JSON

In [48]:
pbar = ntqdm(total=total_pages)
for i in range(total_pages):
    #print(i)
    start_from = i*int(per_page)
    url_to = url_head + str(start_from) + url_middle + per_page + url_tail
    r = S.get(url_to, headers=headers)
    with open(dir+'/page_'+str(i)+'.json', 'w') as f: json.dump(r.json(), f)
    pbar.update(1)

pbar.close()

  0%|          | 0/78 [00:00<?, ?it/s]

TypeError: tqdm_notebook.close() takes 1 positional argument but 2 were given

# Compile the JSONs

In [49]:
all_files = glob.glob(dir+'/*.json')
df_list = []

for filename in all_files:
    with open(filename, 'r') as f:
        dd = json.load(f)['data']
        temp_df = pd.DataFrame(dd)
        df_list.append(temp_df)

In [50]:
df = pd.concat(df_list, ignore_index=True, axis=0)

In [51]:
df

,idBulan,pagu,satuanKerja,isPDN,idlokasi,idKldi,metode,sumberDana,kldi,isUMK,...,id_referensi,lokasi,jenisPengadaan,idSatker,pemilihan,idMetode,id,idJenisPengadaan,paket,pds
0,1,3000000,DINAS LINGKUNGAN HIDUP DAN KEBERSIHAN,True,0,D379,E-Purchasing,APBD,Kab. Indragiri Hilir,True,...,121,"Riau, Indragiri Hilir (Kab.)",Jasa Lainnya,112248,January 2024,9,47767327,4,Belanja Sewa Bangunan Gedung Tempat Pertemuan,False
1,1,6528665,Badan Kepegawaian dan Pengembangan Sumber Daya...,True,0,D459,E-Purchasing,"APBD, APBD, APBD",Kab. Minahasa Selatan,True,...,560,"Sulawesi Utara, Minahasa Selatan (Kab.)",Barang,82468,January 2024,9,47767332,1,Belanja Alat/Bahan untuk Kegiatan Kantor- Kert...,False
2,1,250000,Badan Kesatuan Bangsa dan Politik,True,0,D89,Dikecualikan,APBD,Kota Jambi,True,...,278,"Jambi, Jambi (Kota)",Barang,60977,January 2024,16,47767334,1,Belanja Alat/Bahan untuk Kegiatan Kantor-Benda...,False
3,1,180000000,Sekretariat Daerah,True,0,D399,Pengadaan Langsung,APBD,Kab. Enrekang,True,...,585,"Sulawesi Selatan, Enrekang (Kab.)",Pekerjaan Konstruksi,324031,January 2024,8,47767341,2,Pembuatan papan nama kantor dan perbaikan pint...,False
4,1,7630000,DINAS PERTANIAN,True,0,D239,Pengadaan Langsung,APBD,Kab. Kapuas,True,...,522,"Kalimantan Tengah, Kapuas (Kab.)",Barang,101173,January 2024,8,47767394,1,Belanja Bahan-Bahan Bakar dan Pelumas Sub Kegi...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867842,1,8450000,UPTD Puskesmas Pengasinan,True,0,D114,Pengadaan Langsung,APBD,Kota Bekasi,True,...,374,"Jawa Barat, Bekasi (Kota)",Barang,339035,January 2024,8,50949364,1,Belanja Makan Minum Pengelolaan Surveilans Kes...,False
3867843,1,4880000,DINAS PEMUDA OLAHRAGA DAN PARIWISATA,True,0,D511,Pengadaan Langsung,APBD,Kab. Padang Lawas,True,...,231,"Sumatera Utara, Padang Lawas (Kab.)",Barang,177522,January 2024,8,50949388,1,Belanja Pemeliharaan Alat Kantor dan Rumah Tan...,False
3867844,3,1200000,DINAS PEKERJAAN UMUM DAN PENATAAN RUANG,True,0,D473,Dikecualikan,"APBD, APBD",Kab. Padang Pariaman,True,...,242,"Sumatera Barat, Padang Pariaman (Kab.)",Jasa Lainnya,109132,March 2024,16,50949435,4,Belanja Makanan dan Minuman Rapat,False
3867845,1,3672000,Dinas Pemberdayaan Perempuan & Perlindungan Anak,True,0,D489,E-Purchasing,"APBD, APBD",Kab. Ogan Komering Ilir,True,...,285,"Sumatera Selatan, Ogan Komering Ilir (Kab.)",Barang,174566,January 2024,9,50949451,1,Belanja Alat/Bahan untuk Kegiatan Kantor- Baha...,False


In [53]:
df.to_csv(nama_file+'.csv', index=False, sep='\t')

# Close the browser

In [54]:
DRIVER.quit()

In [56]:
df.head(3).T

,0,1,2
idBulan,1,1,1
pagu,3000000,6528665,250000
satuanKerja,DINAS LINGKUNGAN HIDUP DAN KEBERSIHAN,Badan Kepegawaian dan Pengembangan Sumber Daya...,Badan Kesatuan Bangsa dan Politik
isPDN,True,True,True
idlokasi,0,0,0
idKldi,D379,D459,D89
metode,E-Purchasing,E-Purchasing,Dikecualikan
sumberDana,APBD,"APBD, APBD, APBD",APBD
kldi,Kab. Indragiri Hilir,Kab. Minahasa Selatan,Kota Jambi
isUMK,True,True,True
